# Assignment_5

In [3]:
# We call ChromeDriver
# it allows you to work with differen versions of drivers

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
import time 
import pandas as pd

In [4]:
#Generate an empty data frame to place the other dataframes

driver = webdriver.Chrome()
driver.maximize_window()

dataframes_list = []

url = 'https://infogob.jne.gob.pe/Eleccion'

#Generate a loop to download the information of all elections
#This allows us to group the information corresponding to the elections we want.
for number in range(2,27):
    driver.get(url)
    dropdown = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div')
    dropdown.click()
    time.sleep(1)
    
    presidenciales = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')
    presidenciales.click()
    time.sleep(1)

    eleccion=driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div')
    eleccion.click()
    time.sleep(1)

    election_1 = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[{}]'.format(number))#de 2 a 26
    election_1.click()
    eleccion_name= election_1.text
    time.sleep(1)

    entrar = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button')
    entrar.click()
    time.sleep(2)

    resultado=driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a')
    resultado.click()
    time.sleep(1)

    table_path=driver.find_element(By.ID,'gridEleccionResultadosElectorales')
    table_html = table_path.get_attribute( 'innerHTML' )

    #We create headers to group the information 
    #Add hmtl text, so there are no problems with the read.html
    cabeza="<table id=\"gridEleccionResultadosElectorales\" class=\"table table-hover table-striped\">"
    tabla_new=cabeza+table_html[21:-1]
    tabla_new
    table = pd.read_html(tabla_new)
    tabla_clean=table[ 0 ].iloc[ : , : ]
    tabla_clean = tabla_clean.loc[:, ['ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]
    tabla_clean.insert(0, 'Elecciones', eleccion_name)
    dataframes_list.append(tabla_clean)

In [5]:
#Transform the list of dataframes into a single dataframe
    
df_final = pd.concat(dataframes_list, ignore_index=True)

df_final

Elecciones                 ORGANIZACIÓN POLÍTICA  \
0    PRESIDENCIAL 2021 - 2DA VUELTA  PARTIDO POLITICO NACIONAL PERU LIBRE   
1    PRESIDENCIAL 2021 - 2DA VUELTA                        FUERZA POPULAR   
2    PRESIDENCIAL 2021 - 2DA VUELTA                       VOTOS EN BLANCO   
3    PRESIDENCIAL 2021 - 2DA VUELTA                           VOTOS NULOS   
4                 PRESIDENCIAL 2021  PARTIDO POLITICO NACIONAL PERU LIBRE   
..                              ...                                   ...   
152               PRESIDENCIAL 1936                   PARTIDO REPUBLICANO   
153               PRESIDENCIAL 1931                  UNION REVOLUCIONARIA   
154               PRESIDENCIAL 1931               PARTIDO APRISTA PERUANO   
155               PRESIDENCIAL 1931                PARTIDO DESCENTRALISTA   
156               PRESIDENCIAL 1931            PARTIDO COALICION NACIONAL   

     TOTAL VOTOS  
0        8836380  
1        8792117  
2         121489  
3        1106816  
4        2724752  
..           ...  
152        30803  
153       152149  
154       106088  
155        21950  
156        19640  

[157 rows x 3 columns]

In [6]:
#Export the data to an excel file
#We added a excel file on the ass5 with the content of the elections worked
df_final.to_excel( r'group_4_ass_5_2024.xlsx' , index = False )